### Import

In [ ]:
!pip install category_encoders
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.6 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
import numpy as np
from datetime import datetime
import os
import json
import re

from sklearn.linear_model import (
    LogisticRegression
)

from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier
)

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score
)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import category_encoders as ce

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.stats import rankdata

import warnings
warnings.filterwarnings(action='ignore')

### Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# 현재 작업 디렉터리 출력
print(os.getcwd())

data_path = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'Aimers_6th', 'Data')
print(data_path)

/content
/content/drive/MyDrive/Aimers_6th/Data


In [ ]:
config = {
    'root': data_path
    , 'train_path': f'{data_path}/train.csv'
    , 'test_path': f'{data_path}/test.csv'
    , 'submit_path': f'{data_path}/sample_submission.csv'
    , 'seed_list': [42]
    , 'k_fold': 5
    , 'model': 'lgb'       # cbt, logistic, et, rf, lgb, xgb
    , 'encoding': None # None(lgb, xgb, cbt), target, one-hot, ordinal, catboost

}

### HyperParameter

In [ ]:
# Our best parameters
params_di = {
    'logistic': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'max_iter': 300,
        'penalty': 'l2'
    },

    'et': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'n_estimators': 300,
        'max_depth': 6,
        'min_samples_leaf': 2,
        'max_samples': 0.5

    },

    'rf': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'n_estimators': 300,
        'max_depth': 6,
        'min_samples_leaf': 2,
        # 'max_samples': 0.5,
        'bootstrap': False,

    },

    'lgb': {
        'random_state': config['seed_list'][0],
        'objective': 'binary',
        'n_jobs': 1,
        'verbosity': -1,
        'early_stopping_rounds': 10,
        'deterministic': True,
        'n_estimators': 910,
        'learning_rate': 0.2511885407801018,
        'num_leaves': 28,
        'max_depth': 4,
        'min_child_samples': 56,
        'subsample': 0.10122821600860445,
        'colsample_bytree': 0.6337550995282828,
        'reg_alpha': 0.40455183633441194,
        'reg_lambda': 0.000689949085617777,

    },

    'xgb': {
        'random_state': config['seed_list'][0],
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_jobs': 1,
        'early_stopping_rounds': 10,
        'enable_categorical': True,
        'tree_method': 'hist',

        'n_estimators': 441,
        'learning_rate': 0.03721359333248489,
        'max_depth': 4,
        'min_child_weight': 8.938182919719816,
        'subsample': 0.9671371022238382,
        'colsample_bytree': 0.5780876686670449,
        'gamma': 3.510090814742715e-06,
        'lambda': 1.0496048644425573,
        'alpha': 3.412427838959816e-05,


    },

    # 'cbt': {
    #     'random_seed': config['seed_list'][0],
    #     'objective': 'Logloss',
    #     'eval_metric': 'AUC',
    #     'auto_class_weights': 'Balanced',
    #     'verbose': 100,
    #     'early_stopping_rounds': 10,
    #     'learning_rate': 0.1,
    #     'n_estimators': 300,
    #     'max_depth': 6,
    #     'l2_leaf_reg': 1,
    #     'min_data_in_leaf': 2,
    #     'subsample': 0.5,
    #     'task_type': 'CPU',
    #     'allow_writing_files': False
    # }
    'cbt': {
        'random_seed': config['seed_list'][0],
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'auto_class_weights': 'Balanced',
        'verbose': 100,
        'early_stopping_rounds': 10,
        'task_type': 'CPU',
        'allow_writing_files': False,

        'learning_rate': 0.0051235964245759,
        'n_estimators': 288,
        'max_depth': 9,
        'l2_leaf_reg': 7.513168595258848,
        'bagging_temperature': 0.5106549389945279,
        'random_strength': 5.047046317652829,
        'border_count': 50,
        'min_data_in_leaf': 25,
        'subsample': 0.8799604706872908,
        'colsample_bylevel': 0.7386942588777181,

    }
}

params_ivf = {
    'logistic': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'max_iter': 300,
        'penalty': 'l2'
    },

    'et': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'n_estimators': 300,
        'max_depth': 6,
        'min_samples_leaf': 2,
        'max_samples': 0.5

    },

    'rf': {
        'n_jobs': 1,
        'random_state': config['seed_list'][0],
        'n_estimators': 300,
        'max_depth': 6,
        'min_samples_leaf': 2,
        # 'max_samples': 0.5,
        'bootstrap': False,

    },

    # 'lgb': {
    #     'random_state': config['seed_list'][0],
    #     'objective': 'binary',
    #     'n_jobs': 1,
    #     'verbosity': -1,
    #     'early_stopping_rounds': 10,
    #     'n_estimators': 300,
    #     'learning_rate': 0.1,
    #     'max_depth': 6,
    #     'reg_lambda': 1,
    #     'subsample': 0.5,
    #     'deterministic': True,
    # },

    'lgb': {
        'random_state': config['seed_list'][0],
        'objective': 'binary',
        'n_jobs': 1,
        'verbosity': -1,
        'early_stopping_rounds': 10,
        'deterministic': True,
        'n_estimators': 399,
        'learning_rate': 0.08662393386614221,
        'num_leaves': 30,
        'max_depth': 7,
        'min_child_samples': 9,
        'subsample': 0.8712444912002683,
        'colsample_bytree': 0.7028171113108472,
        'reg_alpha': 3.2093670445847933e-05,
        'reg_lambda': 7.779128043413825,

    },

    'xgb': {
        'random_state': config['seed_list'][0],
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_jobs': 1,
        'early_stopping_rounds': 10,
        'enable_categorical': True,
        'tree_method': 'hist',

        'n_estimators': 237,
        'learning_rate': 0.04776299195908614,
        'max_depth': 6,
        'min_child_weight': 7.627814434191197,
        'subsample': 0.9134763046750406,
        'colsample_bytree': 0.43858126559157823,
        'gamma': 0.013695989699885162,
        'lambda': 0.08441956749102139,
        'alpha': 0.04219273458844118
    },


    # 'cbt': {
    #     'random_seed': config['seed_list'][0],
    #     'objective': 'Logloss',
    #     'eval_metric': 'AUC',
    #     'auto_class_weights': 'Balanced',
    #     'verbose': 100,
    #     'early_stopping_rounds': 10,
    #     'learning_rate': 0.1,
    #     'n_estimators': 300,
    #     'max_depth': 6,
    #     'l2_leaf_reg': 1,
    #     'min_data_in_leaf': 2,
    #     'subsample': 0.5,
    #     'task_type': 'CPU',
    #     'allow_writing_files': False
    # }

    'cbt': {
        'random_seed': config['seed_list'][0],
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'auto_class_weights': 'Balanced',
        'verbose': 100,
        'early_stopping_rounds': 10,
        'task_type': 'CPU',
        'allow_writing_files': False,

        'learning_rate': 0.06420870340097037,
        'n_estimators': 704,
        'max_depth': 7,
        'l2_leaf_reg': 5.428570600396231,
        'bagging_temperature': 0.9964805026288809,
        'random_strength': 1.4950577092436823,
        'border_count': 109,
        'min_data_in_leaf': 77,
        'subsample': 0.42364067415772133,
        'colsample_bylevel': 0.5429603138618737,
    }
}

### Function

In [ ]:
def set_seed(seed: int): # seed 고정 함수
    # Set the seed for reproducibility.
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


def read_data(config):
    # Load training, testing, and submission CSV files
    df_train = pd.read_csv(config['train_path']).drop(columns=['ID'])  # train data
    df_test = pd.read_csv(config['test_path']).drop(columns=['ID'])    # test data
    df_sub = pd.read_csv(config['submit_path'])

    print(f'train data 수: {df_train.shape[0]}')
    print(f'test data 수: {df_test.shape[0]}')
    print(f'submission data 수: {df_sub.shape[0]}')
    return df_train, df_test, df_sub

def df_ivf_di_split(df, is_train=False):

    data_type = 'train' if is_train else 'test'

    print(f'data 수: {df.shape[0]}')

    df_ivf = df[df['시술 유형'] == 'IVF'].drop(columns=['시술 유형'])
    df_di = df[df['시술 유형'] == 'DI'].drop(columns=['시술 유형'])

    print(f'시술유형 IVF {data_type} data 수: {df_ivf.shape[0]}')
    print(f'시술유형 DI {data_type} data 수: {df_di.shape[0]}')

    return df_ivf, df_di


def get_clf_eval(y_test, y_proba=None, fold_no=None):
    # Calculate and print evaluation metrics and confusion matrix,
    # accuracy, precision, recall, f1 and roc_auc score.
    # Optionally includes fold number in the output.

    # 임계값 0.5 기준 예측값 생성
    y_pred = (y_proba >= 0.5).astype(int)

    y_test = y_test.values

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)  # ROC-AUC는 확률값 그대로 사용

    fold_info = f'Fold #{fold_no}' if fold_no is not None else ''
    print(f'{fold_info} ACC: {accuracy:.4f}, PRE: {precision:.4f}, REC: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}')
    return roc_auc


# Categorical variable encoding method.
def category_encoding(df_train, df_test, base_num_features, base_cat_features, config, is_ivf=False):
    target = '임신 성공 여부'

    data_name = 'IVF' if is_ivf else 'DI'

    new_df_train = pd.concat([df_train[base_num_features].copy(), df_train[target]], axis = 1)
    new_df_test = df_test[base_num_features].copy()

    print(f'{data_name} category 변수 인코딩: {config["encoding"]}')

    if config['encoding'] == 'target':
        encoder = ce.TargetEncoder(cols=base_cat_features)

    elif config['encoding'] == 'ordinal':
        encoder = ce.OrdinalEncoder(cols=base_cat_features)

    elif config['encoding'] == 'catboost':
        encoder = ce.CatBoostEncoder(cols=base_cat_features)#, random_state=config['seed'])

    if config['encoding'] in ['target', 'ordinal', 'catboost']:
        encoder.fit(df_train[base_cat_features], df_train[target])
        new_df_train[base_cat_features] = encoder.transform(df_train[base_cat_features])
        new_df_test[base_cat_features] = encoder.transform(df_test[base_cat_features])

    elif config['encoding'] == 'one-hot':
        encoder = ce.OneHotEncoder(cols=base_cat_features, use_cat_names = True)
        encoder.fit(df_train[base_cat_features], df_train[target])
        result_tr = encoder.transform(df_train[base_cat_features])
        result_te = encoder.transform(df_test[base_cat_features])

        result_tr.columns = result_tr.columns.str.replace(r'[^ㄱ-ㅎ가-힣A-Za-z0-9_]', '_', regex=True)
        result_te.columns = result_te.columns.str.replace(r'[^ㄱ-ㅎ가-힣A-Za-z0-9_]', '_', regex=True)

        new_df_train = pd.concat([new_df_train, result_tr], axis = 1)
        new_df_test = pd.concat([new_df_test, result_te], axis = 1)

    elif config['encoding'] is None:
        new_df_train = pd.concat([new_df_train, df_train[base_cat_features]], axis = 1)
        new_df_test = pd.concat([new_df_test, df_test[base_cat_features]], axis = 1)

    _, base_cat_features, base_features = make_feature_lists(new_df_train, is_ivf=is_ivf)

    return new_df_train, new_df_test, base_cat_features, base_features

def feature_engineering(df_input, tr_removed_column_list, is_train=False, is_ivf=False):
    df = df_input.copy()

    data_name = 'IVF' if is_ivf else 'DI'
    data_type = 'train' if is_train else 'test'

    if is_train:
        removed_column_list = []
    else:
        removed_column_list = tr_removed_column_list


    print(f'중복 제거 전 {data_type} {data_name} data 수: {df.shape[0]}')

    # drop_duplicates
    df = df.drop_duplicates(keep='first')

    print(f'중복 제거 후 {data_type} {data_name} data 수: {df.shape[0]}')

    # 횟수 관련 column
    count_columns = [column for column in df.columns if '횟수' in column]

    for col in count_columns:
        df[col] = df[col].str.replace('회', '').str.replace('이상', '').str.replace(' ', '').astype(int)


    # 시술 당시 나이 범위를 중앙값으로 변환하는 매핑 딕셔너리 (누락된 범위 추가)
    treatment_age = {
        "만18-34세": 26,
        "만35-37세": 36,
        "만38-39세": 38.5,
        "만40-42세": 41,
        "만43-44세": 43.5,
        "만45-50세": 47.5,
        "알 수 없음": -1  # 알 수 없음은 -1로 처리
    }

    # 공백 제거 후 매핑
    # 시술 당시 나이 숫자 mapping -> DI에만 적용
    if is_ivf:
        df["시술 당시 나이"] = df["시술 당시 나이"].str.strip().map(treatment_age).astype(float)

    # # 정자 난자기증자 나이 범위를 중앙값으로 변환하는 매핑 딕셔너리 (누락된 범위 추가)
    donor_age = {
        "만20세 이하": 20,
        "만21-25세": 23,
        "만26-30세": 28,
        "만31-35세": 33,
        "만36-40세": 38,
        "만41-45세": 43,
        "알 수 없음": -1 # 알 수 없음은 -1로 처리
    }

    # 공백 제거 후 매핑
    if is_ivf:
        df["난자 기증자 나이"] = df["난자 기증자 나이"].str.strip().map(donor_age).astype(float)

    df["정자 기증자 나이"] = df["정자 기증자 나이"].str.strip().map(donor_age).astype(float)

    # 특정 시술 유형
    treat_types = []
    # embryo_creation_reasons = []

    if not is_ivf:
        treat_types.extend(['IUI', 'ICI', 'Generic DI', 'IVI'])

    if is_ivf:
        treat_types.extend(['ICSI', 'IVF', 'BLASTOCYST', 'AH', 'Unknown'])
        # embryo_creation_reasons.extend(['현재 시술용', '배아 저장용', '기증용', '난자 저장용'])

    # 각 target에 대한 Multi-Hot Encoding 및 Count 계산
    for treat_type in treat_types:
        # 각 행에서 target이 등장하는 횟수 계산
        df[treat_type] = df['특정 시술 유형'].str.count(treat_type)

    # if is_ivf:
    #     for embryo_creation_reason in embryo_creation_reasons:
    #         # 각 행에서 target이 등장하는 횟수 계산
    #         df[embryo_creation_reason] = df['배아 생성 주요 이유'].str.count(embryo_creation_reason)

    df_column_list = df.columns
    print(f'column 제거 전 {data_type} {data_name} data shape: {df.shape}')

    if is_train:
        # train data에서 전부 결측치거나 값 1개만 갖는 column 제거

        for feat in df_column_list:

            null_count = df[feat].isna().sum() # feat 열 결측치 수

            if df[feat].nunique() == 0 or (df[feat].nunique() == 1 and null_count == 0):
                df = df.drop(columns=[feat])
                removed_column_list.append(feat)


    # # test data에서는 train data에서 제거된 column 제거
    else:

        df = df.drop(columns=removed_column_list)

    print(f'column 제거 후 {data_type} {data_name} data shape: {df.shape}')

    # 파생변수 생성
    # IVF에만 존재하는 피처
    if is_ivf:
        # 비율 및 비율 차이
        df["배아_생성률"] = df["미세주입에서 생성된 배아 수"] / (df["미세주입된 난자 수"] + 1e-5)
        df["배아_이식률"] = df["이식된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["미세주입_이식률"] = df["미세주입 배아 이식 수"] / (df["미세주입에서 생성된 배아 수"] + 1e-5)
        df["배아_저장률"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["배아_해동률"] = df["해동된 배아 수"] / (df["저장된 배아 수"] + 1e-5)
        df["기증자_정자_비율"] = df["기증자 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
        df["배아_손실률"] = 1 - (df["미세주입에서 생성된 배아 수"] / (df["미세주입된 난자 수"] + 1e-5))
        df["배아_이식_대비_해동된_배아_비율"] = df["해동된 배아 수"] / (df["이식된 배아 수"] + 1e-5)
        df["미세주입_비율"] = df["미세주입된 난자 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["신선_난자_저장률"] = df["저장된 신선 난자 수"] / (df["수집된 신선 난자 수"] + 1e-5)
        df["정자_혼합_비율_차이"] = (
            df["파트너 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
            - df["기증자 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
        )

        # 차이 및 변화량
        df["미세주입_실패_수"] = df["미세주입된 난자 수"] - df["미세주입에서 생성된 배아 수"]
        df["이식되지_않은_배아_수"] = df["미세주입에서 생성된 배아 수"] - df["미세주입 배아 이식 수"]
        df["저장되지_않은_신선_난자_수"] = df["수집된 신선 난자 수"] - df["저장된 신선 난자 수"]
        df["기증자_혼합_난자_수"] = df["혼합된 난자 수"] - df["파트너 정자와 혼합된 난자 수"]
        df["해동_후_이식_까지_시간"] = df["배아 이식 경과일"] - df["배아 해동 경과일"]
        df["해동_후_미세주입_성공률"] = df["미세주입된 난자 수"] / (df["해동된 배아 수"] + 1e-5)
        # df["배아_생성_감소량"] = df["미세주입된 난자 수"] - df["미세주입에서 생성된 배아 수"]
        # df["배아_해동_후_남은_기간"] = df["배아 이식 경과일"] - df["배아 해동 경과일"]
        df["배아_저장_후_해동_기간"] = df["배아 해동 경과일"] - df["난자 혼합 경과일"]
        df["정자_혼합_후_배아_생성_경과일"] = df["배아 이식 경과일"] - df["난자 혼합 경과일"]
        # df["미세주입_대비_해동된_배아_비율"] = df["미세주입된 난자 수"] / (df["해동된 배아 수"] + 1e-5)

    # 불임 원인 multi hot -> 하나의 feature

    # infertility_columns = [column for column in df.columns if '불임 원인' in column]

    # # dot()을 사용하여 1이 있는 컬럼을 결합 (각 feature 사이에 공백 추가)
    # df['불임 원인 종합'] = df[infertility_columns].dot(pd.Index([f'{col.replace("불임 원인", "").replace(" ", "")} ' for col in infertility_columns])).str.strip()

    # # 한글만 남기고 나머지 문자 제거 (정규 표현식 사용)
    # df['불임 원인 종합'] = df['불임 원인 종합'].apply(lambda x: re.sub(r'[^가-힣 ]', '', x))

    # # split()과 join()을 사용하여 공백을 ','로 변경
    # df['불임 원인 종합'] = df['불임 원인 종합'].apply(lambda x: ', '.join(x.split()))

    # df = df.drop(columns=infertility_columns)

    return df, removed_column_list

def make_feature_lists(df, is_ivf=False):
    base_features = []     # all features except target variable.
    base_num_features = [] # numerical features
    base_cat_features = [] # categorical features

    data_name = 'IVF' if is_ivf else 'DI'

    for feat in df.columns:
        # skip the target
        if feat == '임신 성공 여부':
            continue

        base_features.append(feat)

        if df[feat].dtype in ['object', 'category']:
            base_cat_features.append(feat)
        else:
            base_num_features.append(feat)

    # infertility_columns = [column for column in df.columns if '불임 원인' in column]

    # 공통 제거 변수
    removal_features = {
            'ID', '불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 자궁경부 문제',
            '시술 유형', '배란 유도 유형', '특정 시술 유형','배란 자극 여부','부부 주 불임 원인','여성 주 불임 원인'}

    # ivf 제거 변수
    removal_features_ivf = {
        '남성 주 불임 원인', '남성 부 불임 원인','부부 부 불임 원인','AH','신선_난자 저장률','IVF',
        '저장된 신선 난자 수', '신선 배아 사용 여부', '대리모 여부','난자 해동 경과일','DI 임신 횟수','불임 원인 - 정자 농도',
        '동결 배아 사용 여부','배아_저장_후_해동_기간','불임 원인 - 정자 운동성','DI 출산 횟수',
        '불임 원인 - 정자 형태', '기증 배아 사용 여부','배아 해동 경과일'
    }

    # di 제거 변수
    removal_features_di = {
        '불임 원인- 배란 장애','불명확 불임 원인',' 임신 시도 또는 마지막 임신 경과 연수','총 출산 횟수',
        '총 임신 횟수','시술 시기 코드'
    }
    if is_ivf:
        removal_features = removal_features.union(removal_features_ivf)
    else:
        removal_features = removal_features.union(removal_features_di)

    # removal_features.update(infertility_columns)

    # remove the specified features
    base_num_features = [i for i in base_num_features if i not in removal_features]
    base_cat_features = [i for i in base_cat_features if i not in removal_features]
    base_features = [i for i in base_features if i not in removal_features]

    print(f'{data_name} numeric feature 수: {len(base_num_features)}')
    print(f'{data_name} category feature 수: {len(base_cat_features)}')
    print(f'{data_name} 전체 feature 수: {len(base_features)}')

    return base_num_features, base_cat_features, base_features

def filling_missing_values(df_input, base_cat_features, base_num_features, config):
    df = df_input.copy()

    # Fill missing values for categorical features with 'Unknown' and ensure their data type is string.
    for base_cat_feat in base_cat_features:
        df[base_cat_feat] = df[base_cat_feat].astype(str)
        df[base_cat_feat] = df[base_cat_feat].fillna('알 수 없음')

        if config['encoding'] is None:
            df[base_cat_feat] = df[base_cat_feat].astype('category')


    # Fill missing values for numerical features with -1.
    for base_num_feat in base_num_features:
        df[base_num_feat] = df[base_num_feat].fillna(-1)

    return df


def model_kfold(df, config, params, base_features, base_cat_features, is_ivf=False):
    target = '임신 성공 여부'

    data_name = 'IVF' if is_ivf else 'DI'

    skf = StratifiedKFold(n_splits=config['k_fold'], shuffle=True, random_state=config['seed'])
    models = []       # trained models
    roc_auc_scores = []  # roc_auc_scores for validation sets

    model_params = params[config['model']]

    for k_fold, (train_idx, valid_idx) in enumerate(skf.split(df[base_features], df[target])):
        print(f'Fold #{k_fold + 1}')
        X_train, y_train = df[base_features].iloc[train_idx], df[target].iloc[train_idx]
        X_valid, y_valid = df[base_features].iloc[valid_idx], df[target].iloc[valid_idx]

        if config['model'] == 'logistic':
            model = LogisticRegression(**model_params)

        elif config['model'] == 'et':
            model = ExtraTreesClassifier(**model_params)

        elif config['model'] == 'rf':
            model = RandomForestClassifier(**model_params)

        if config['model'] in ['logistic', 'et', 'rf']:
            model.fit(X_train, y_train)

        elif config['model'] == 'lgb':
            model = LGBMClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric='auc',
                categorical_feature=base_cat_features, # specify categorical features
            )

        elif config['model'] == 'xgb':
            model = XGBClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                verbose = 100
            )

        elif config['model'] == 'cbt':
            model = CatBoostClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                cat_features=base_cat_features, # specify categorical features
            )

        # save the trained model
        models.append(model)

        # evaluate the model
        # --- train-set
        print(f'[{data_name} Train] ', end='')
        y_prob = model.predict_proba(X_train)[:, 1]
        _ = get_clf_eval(y_train, y_prob, k_fold + 1)

        # --- valid-set
        print(f'[{data_name} Valid] ', end='')
        y_prob = model.predict_proba(X_valid)[:, 1]
        roc_auc_score = get_clf_eval(y_valid, y_prob, k_fold + 1)

        roc_auc_scores.append(roc_auc_score)

    avg_roc_auc = np.mean(roc_auc_scores)
    var_roc_auc = np.var(roc_auc_scores)

    print(f'Avg. roc-auc of validset {data_name}: {avg_roc_auc}')
    print(f'Var. roc-auc of validset {data_name}: {var_roc_auc}')

    return models, avg_roc_auc, var_roc_auc

def kfold_submission(df_test_ivf, df_test_di, df_sub, models_ivf, models_di, config):
    feat_importance_path = f"{config['root']}/FeatureImportance"
    submission_path = f"{config['root']}/Submission"
    json_path = f"{config['root']}/Json"

    if not os.path.exists(feat_importance_path):
        os.makedirs(feat_importance_path)

    if not os.path.exists(submission_path):
        os.makedirs(submission_path)

    if not os.path.exists(json_path):
        os.makedirs(json_path)

    # get current date and time
    now = datetime.now()

    # record the year, month, day, hour, and minute for naming files.
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute

    # file format
    submission_time = f"{year:04d}{month:02d}{day:02d}_{hour:02d}{minute:02d}"[2:]
    target = 'probability'

    # apply feature engineering
    base_num_features_ivf, base_cat_features_ivf, base_features_ivf = make_feature_lists(df_test_ivf, is_ivf=True)
    df_test_ivf = filling_missing_values(df_test_ivf, base_cat_features_ivf, base_num_features_ivf, config)
    X_test_ivf = df_test_ivf[base_features_ivf]

    base_num_features_di, base_cat_features_di, base_features_di = make_feature_lists(df_test_di, is_ivf=False)
    df_test_di = filling_missing_values(df_test_di, base_cat_features_di, base_num_features_di, config)
    X_test_di = df_test_di[base_features_di]

    # dataframe for feature importances
    base_features = list(set(base_features_ivf).union(set(base_features_di)))

    df_feature_importance_all_ivf = pd.DataFrame({'features': base_features_ivf})
    df_feature_importance_all_di = pd.DataFrame({'features': base_features_di})

def rank_ensemble(predictions_list):
    """
    여러 모델의 예측값을 받아서 Rank Ensemble을 적용한 최종 예측값을 반환하는 함수
    """
    num_models = len(predictions_list)
    num_samples = len(predictions_list[0])

    rank_preds = np.zeros((num_samples, num_models))

    # 각 모델의 예측값을 랭크 변환
    for i, preds in enumerate(predictions_list):
        rank_preds[:, i] = rankdata(preds) / num_samples  # 0~1 범위로 정규화된 순위

    # 모든 모델의 Rank 평균
    final_rank_preds = rank_preds.mean(axis=1)

    return final_rank_preds

    ivf_preds_list = []
    di_preds_list = []

    for i, model_ivf in enumerate(models_ivf):
      preds_ivf = model_ivf.predict_proba(X_test_ivf)[:, 1]
      ivf_preds_list.append(preds_ivf)


      # save feature importance of current model
      if config['model'] in ['logistic']:
        df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.coef_.squeeze()

      elif config['model'] in ['et', 'rf', 'lgb', 'xgb']:
        df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.feature_importances_

      elif config['model'] == 'cbt':
        df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.get_feature_importance()

    for i, model_di in enumerate(models_di):
      preds_di = model_di.predict_proba(X_test_di)[:, 1]
      di_preds_list.append(preds_di)

    # save feature importance of current model
      if config['model'] in ['logistic']:
        df_feature_importance_all_di[f'model_{i}'] = model_di.coef_.squeeze()

      elif config['model'] in ['et', 'rf', 'lgb', 'xgb']:
        df_feature_importance_all_di[f'model_{i}'] = model_di.feature_importances_

      elif config['model'] == 'cbt':
        df_feature_importance_all_di[f'model_{i}'] = model_di.get_feature_importance()

    # Rank Ensemble 적용
    y_probs_ivf = rank_ensemble(ivf_preds_list)
    y_probs_di = rank_ensemble(di_preds_list)

    # 최종 예측값 저장
    df_sub.loc[df_test_ivf.index, target] = y_probs_ivf
    df_sub.loc[df_test_di.index, target] = y_probs_di

    # save submission file as CSV
    df_sub.to_csv(f"{submission_path}/{submission_time}_{config['model']}_{config['encoding']}_submission.csv", index=False)

    # compute avarege, rank
    df_feature_importance_all_ivf['average'] = df_feature_importance_all_ivf.iloc[:, 1:].mean(axis=1).values
    df_feature_importance_all_ivf['rank'] = df_feature_importance_all_ivf['average'].rank(ascending=False)
    df_feature_importance_all_ivf = df_feature_importance_all_ivf.sort_values(by='rank')

    df_feature_importance_all_di['average'] = df_feature_importance_all_di.iloc[:, 1:].mean(axis=1).values
    df_feature_importance_all_di['rank'] = df_feature_importance_all_di['average'].rank(ascending=False)
    df_feature_importance_all_di = df_feature_importance_all_di.sort_values(by='rank')

    # save the feature importance as CSV
    df_feature_importance_all_ivf.to_csv(f'{feat_importance_path}/feat_import_{submission_time}_{config["model"]}_IVF_{config["encoding"]}.csv', index=False)
    df_feature_importance_all_di.to_csv(f'{feat_importance_path}/feat_import_{submission_time}_{config["model"]}_DI_{config["encoding"]}.csv', index=False)

    # save parameters as JSON
    json_data = json.dumps(config, indent=4)

    with open(f'{json_path}/{submission_time}_{config["model"]}_{config["encoding"]}.json', 'w') as file:
        file.write(json_data)

### Data Pre-processing

In [ ]:
def main(config, params_di, params_ivf):
    models_ivf = []
    models_di = []

    for seed in config['seed_list']:
        config['seed'] = seed
        params_di['random_seed'] = seed
        params_ivf['random_seed'] = seed

        # set seed
        set_seed(config['seed'])

        # read data set
        df_train, df_test, df_sub = read_data(config)

        # ivf di split
        df_train_ivf, df_train_di = df_ivf_di_split(df_train, is_train = True)
        df_test_ivf, df_test_di = df_ivf_di_split(df_test, is_train = False)

        # feature engineering (train,test)
        df_train_ivf, removed_column_list_ivf = feature_engineering(df_train_ivf, None, is_train=True, is_ivf=True)
        df_test_ivf, _ = feature_engineering(df_test_ivf, removed_column_list_ivf, is_train=False, is_ivf=True)

        df_train_di, removed_column_list_di = feature_engineering(df_train_di, None, is_train=True, is_ivf=False)
        df_test_di, _ = feature_engineering(df_test_di, removed_column_list_di, is_train=False, is_ivf=False)

        # feature list 생성 (train)
        base_num_features_ivf, base_cat_features_ivf, base_features_ivf = make_feature_lists(df_train_ivf, is_ivf=True)
        base_num_features_di, base_cat_features_di, base_features_di = make_feature_lists(df_train_di, is_ivf=False)

        # 결측치 처리
        df_train_ivf = filling_missing_values(df_train_ivf, base_cat_features_ivf, base_num_features_ivf, config)
        df_train_di = filling_missing_values(df_train_di, base_cat_features_di, base_num_features_di, config)

        # encoding
        df_train_ivf, df_test_ivf, base_cat_features_ivf, base_features_ivf = category_encoding(df_train_ivf, df_test_ivf, base_num_features_ivf, base_cat_features_ivf, config, is_ivf=True)
        df_train_di, df_test_di, base_cat_features_di, base_features_di = category_encoding(df_train_di, df_test_di, base_num_features_di, base_cat_features_di, config, is_ivf=False)

        # check model performance
        model_ivf, avg_roc_auc_ivf, var_roc_auc_ivf = model_kfold(df_train_ivf, config, params_ivf, base_features_ivf, base_cat_features_ivf, is_ivf=True)
        model_di, avg_roc_auc_di, var_roc_auc_di = model_kfold(df_train_di, config, params_di, base_features_di, base_cat_features_di, is_ivf=False)

        config['avg_roc_auc_ivf'] = avg_roc_auc_ivf
        config['var_roc_auc_ivf'] = var_roc_auc_ivf

        config['avg_roc_auc_di'] = avg_roc_auc_di
        config['var_roc_auc_di'] = var_roc_auc_di

        models_ivf.extend(model_ivf)
        models_di.extend(model_di)

    config['model_param_di'] = params_di[config['model']]
    config['model_param_ivf'] = params_ivf[config['model']]

    # submissionㅊ
    kfold_submission(df_test_ivf, df_test_di, df_sub, models_ivf, models_di, config)

In [ ]:
main(config, params_di, params_ivf)

train data 수: 256351
test data 수: 90067
submission data 수: 90067
data 수: 256351
시술유형 IVF train data 수: 250060
시술유형 DI train data 수: 6291
data 수: 90067
시술유형 IVF test data 수: 87891
시술유형 DI test data 수: 2176
중복 제거 전 train IVF data 수: 250060
중복 제거 후 train IVF data 수: 250060
column 제거 전 train IVF data shape: (250060, 72)
column 제거 후 train IVF data shape: (250060, 71)
중복 제거 전 test IVF data 수: 87891
중복 제거 후 test IVF data 수: 87891
column 제거 전 test IVF data shape: (87891, 71)
column 제거 후 test IVF data shape: (87891, 70)
중복 제거 전 train DI data 수: 6291
중복 제거 후 train DI data 수: 6291
column 제거 전 train DI data shape: (6291, 71)
column 제거 후 train DI data shape: (6291, 34)
중복 제거 전 test DI data 수: 2176
중복 제거 후 test DI data 수: 2176
column 제거 전 test DI data shape: (2176, 70)
column 제거 후 test DI data shape: (2176, 33)
IVF numeric feature 수: 60
IVF category feature 수: 4
IVF 전체 feature 수: 64
DI numeric feature 수: 24
DI category feature 수: 1
DI 전체 feature 수: 25
IVF category 변수 인코딩: None
IVF numeric feature 수:

In [ ]:
config

{'root': '/content/drive/MyDrive/Aimers_6th/Data',
 'train_path': '/content/drive/MyDrive/Aimers_6th/Data/train.csv',
 'test_path': '/content/drive/MyDrive/Aimers_6th/Data/test.csv',
 'submit_path': '/content/drive/MyDrive/Aimers_6th/Data/sample_submission.csv',
 'seed_list': [42],
 'k_fold': 5,
 'model': 'lgb',
 'encoding': None,
 'seed': 42,
 'avg_roc_auc_ivf': 0.7383214411693837,
 'var_roc_auc_ivf': 1.8850283281632053e-06,
 'avg_roc_auc_di': 0.6922113442200428,
 'var_roc_auc_di': 0.0007523959974091739,
 'model_param_di': {'random_state': 42,
  'objective': 'binary',
  'n_jobs': 1,
  'verbosity': -1,
  'early_stopping_rounds': 10,
  'deterministic': True,
  'n_estimators': 910,
  'learning_rate': 0.2511885407801018,
  'num_leaves': 28,
  'max_depth': 4,
  'min_child_samples': 56,
  'subsample': 0.10122821600860445,
  'colsample_bytree': 0.6337550995282828,
  'reg_alpha': 0.40455183633441194,
  'reg_lambda': 0.000689949085617777},
 'model_param_ivf': {'random_state': 42,
  'objective'

In [ ]:
# read data set
df_train, df_test, df_sub = read_data(config)

# ivf di split
df_train_ivf, df_train_di = df_ivf_di_split(df_train, is_train = True)
df_test_ivf, df_test_di = df_ivf_di_split(df_test, is_train = False)

train data 수: 256351
test data 수: 90067
submission data 수: 90067
data 수: 256351
시술유형 IVF train data 수: 250060
시술유형 DI train data 수: 6291
data 수: 90067
시술유형 IVF test data 수: 87891
시술유형 DI test data 수: 2176


In [ ]:
unique_counts = {column: df_train_ivf[column].nunique() for column in df_train_ivf.columns}
for column, count in unique_counts.items():
    print(f"{column}: {count}")

In [ ]:
unique_counts = {column: df_train_di[column].nunique() for column in df_train_di.columns}
for column, count in unique_counts.items():
    print(f"{column}: {count}")

시술 시기 코드: 7
시술 당시 나이: 6
임신 시도 또는 마지막 임신 경과 연수: 19
특정 시술 유형: 5
배란 자극 여부: 2
배란 유도 유형: 1
단일 배아 이식 여부: 0
착상 전 유전 검사 사용 여부: 0
착상 전 유전 진단 사용 여부: 0
남성 주 불임 원인: 2
남성 부 불임 원인: 2
여성 주 불임 원인: 2
여성 부 불임 원인: 2
부부 주 불임 원인: 2
부부 부 불임 원인: 2
불명확 불임 원인: 2
불임 원인 - 난관 질환: 2
불임 원인 - 남성 요인: 2
불임 원인 - 배란 장애: 2
불임 원인 - 여성 요인: 1
불임 원인 - 자궁경부 문제: 1
불임 원인 - 자궁내막증: 2
불임 원인 - 정자 농도: 2
불임 원인 - 정자 면역학적 요인: 1
불임 원인 - 정자 운동성: 1
불임 원인 - 정자 형태: 2
배아 생성 주요 이유: 0
총 시술 횟수: 7
클리닉 내 총 시술 횟수: 7
IVF 시술 횟수: 7
DI 시술 횟수: 7
총 임신 횟수: 6
IVF 임신 횟수: 4
DI 임신 횟수: 6
총 출산 횟수: 4
IVF 출산 횟수: 3
DI 출산 횟수: 4
총 생성 배아 수: 0
미세주입된 난자 수: 0
미세주입에서 생성된 배아 수: 0
이식된 배아 수: 0
미세주입 배아 이식 수: 0
저장된 배아 수: 0
미세주입 후 저장된 배아 수: 0
해동된 배아 수: 0
해동 난자 수: 0
수집된 신선 난자 수: 0
저장된 신선 난자 수: 0
혼합된 난자 수: 0
파트너 정자와 혼합된 난자 수: 0
기증자 정자와 혼합된 난자 수: 0
난자 출처: 1
정자 출처: 1
난자 기증자 나이: 1
정자 기증자 나이: 7
동결 배아 사용 여부: 0
신선 배아 사용 여부: 0
기증 배아 사용 여부: 0
대리모 여부: 0
PGD 시술 여부: 0
PGS 시술 여부: 0
난자 채취 경과일: 0
난자 해동 경과일: 0
난자 혼합 경과일: 0
배아 이식 경과일: 0
배아 해동 경과일: 0
임신 성공 여부: 2
